# SWOT denoising module: description and use

* Emmanuel Cosme, Laura Gómez Navarro, Julien Le Sommer, Nicolas Papadakis, Ananda Pascual, Audrey Monsimer
* Objective: Describe the denoising methods and show how to call them

### History

* March 2018: creation
* January 2020: conda environment (A. Albert)
* April 2023: bilateral filter (M. A. Abdulfatai)


In [ ]:
import time

print('Last update:', time.ctime(time.time())) 


### Conda environment info

Use of a conda environment configuration file : env_swotmod.yml
```conda env create -f env_swotmod.yml```
then
```conda activate swotmod```

To make it appear on the list of kernels proposed by jupyter :

```python -m ipykernel install --user --name swotmod --display swotmod```

All the libraries needeed ara now installed !

## 0. Imports, versions, plots, input files

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr

import SWOTdenoise as swotd

# Libraries needed in SWOT module
import netCDF4
import scipy

%load_ext autoreload
%autoreload 2

###  A function for plots
This function plots a SSH field as read in a SWOT file.
The function calls the fill_nadir_gap function of the SWOTdenoise module, to clearly show the gap. Otherwise, pyplot fills the gap by crudely interpolating the data between the two swaths.

In [ ]:
vmin = -0.3
vmax = 0.15

def splot(sshin):
    _, lon, lat, x_ac, time = swotd.read_data(filename, 'SSH_obs', 'lon', 'lat', 'x_ac', 'time')
    ssh_f, lon_f, lat_f, x_ac_f = swotd.fill_nadir_gap(sshin, lon, lat, x_ac, time)
    cs = plt.pcolormesh(lon_f, lat_f, ssh_f, vmin=vmin, vmax=vmax)
    plt.colorbar(cs)
    
def splot_inpainted(sshin, lonin, latin):
    cs = plt.pcolormesh(lonin, latin, sshin, vmin=vmin, vmax=vmax)
    plt.colorbar(cs)

### Inputs

In [ ]:
filedir  = 'example_data/'
filename = filedir + 'MED_fastPhase_1km_swotFAST_c01_p009.nc'

## 1. Call the _SWOTdenoise_ function

### 1.1 Using a file name

The ```SWOTdenoise``` function can be called by providing the SWOT Netcdf file name only. If no output file name is given in argument, the denoised field is returned in another Netcdf file named **foo_denoised.nc** if the input file is **foo.nc**. It is created in the same directory.

In [ ]:
output_filename = 'None'
swotd.SWOTdenoise(filename, output_filename)

#### Plot raw initial (left) and denoised (right) SSH

In [ ]:
## Load initial SSH
ssh, lon, lat, x_ac, time = swotd.read_data(filename, 'SSH_obs', 'lon', 'lat', 'x_ac', 'time')

## Load denoised SSH
filename_den = filedir + 'MED_fastPhase_1km_swotFAST_c01_p009_denoised.nc'
ssh_den1, lon, lat, x_ac, time = swotd.read_data(filename_den, 'SSH', 'lon', 'lat', 'x_ac', 'time')

## Plots
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
splot(ssh)
plt.subplot(1,2,2)
splot(ssh_den1)

### 1.2 Using input arrays

The ```SWOTdenoise``` function can also be used by providing the appropriate arrays as keywords arguments: ```ssh, lon, lat, x_ac, time```. The denoised SSH is then returned. In the example below, these arrays are first read in the SWOT file using the ```read_data``` function of the module:

In [ ]:
ssh_den2 = swotd.SWOTdenoise(ssh=ssh, lon=lon, lat=lat, x_ac=x_ac, time=time)

#### Plot raw initial (left) and denoised (right) SSH

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
splot(ssh)
plt.subplot(1,2,2)
splot(ssh_den2)

## 2. Denoising methods and optional parameters

In addition to the arguments described above, the ```SWOTdenoise``` function takes 5 optional arguments:

- ```method```: ```gaussian```, ```boxcar```, , ```bilateral_filter``` or ```var_reg_fista```;
- ```param```: number for ```gaussian``` and ```boxcar```; 2-entry tuple for ```bilateral_filter```; 3-entry tuple for ```var_reg_fista```;
- ```itermax```: only for ```var_reg_fista```: maximum number of iterations in the gradient descent algortihm;
- ```epsilon```: only for ```var_reg_fista```: convergence criterium for the gradient descent algortihm;
- ```inpainting```: if ```True```, the nadir gap is inpainted. If ```False```, it is not and the returned SSH array is of the same shape as the original one. If the ```SWOTdenoise``` function is called using arrays (see above description) with ```inpainting=True```, then it returns SSH, lon, and lat arrays. If it is called using arrays with ```inpainting=False```, it returns only SSH, since lon and lat arrays are the same as for the input field.

Default is the varitional regularization method with prescribed parameters (0, 10, 0) and no inpainting.

Examples are given below.

In [ ]:
ssh, lon, lat, x_ac, time = swotd.read_data(filename, 'SSH_obs', 'lon', 'lat', 'x_ac', 'time')

In [ ]:
splot(ssh)

### Gaussian filter

In [ ]:
ssh_gau = swotd.SWOTdenoise(ssh=ssh, lon=lon, lat=lat, x_ac=x_ac, time=time, method='gaussian', param=10.)
splot(ssh_gau)


### Boxcar filter:

In [ ]:
ssh_box = swotd.SWOTdenoise(ssh=ssh, lon=lon, lat=lat, x_ac=x_ac, time=time, method='boxcar', param=4.)
splot(ssh_box)

### Variational regularization filter

The method uses the second-order term (laplacian) only, with parameter 4, a maximum of 500 iterations and fills the nadir gap with inpainting.

The function prints the number of iterations of the gradient descent method, and the convergence ratio. The convergence ratio is defined here as the ratio between the norm of the descent increment and the convergence factor (equal to 1e-9 by default). Iterations stops when the convergence ratio is below 1.

In [ ]:
ssh_vr2, lon_vr2, lat_vr2 = swotd.SWOTdenoise(ssh=ssh, lon=lon, lat=lat, x_ac=x_ac, time=time, method='var_reg_fista',
                                              param=(0,4,0), itermax = 500, inpainting=True)
splot_inpainted(ssh_vr2, lon_vr2, lat_vr2)

### Bilateral filter

This takes several minutes on a laptop. The code requires optimization.

The first parameter is the standard deviation in pixels for the (gaussian) spatial filter. The second one is the standard deviation in SSH units (m) for the photometric filter.

A good reference is: 10.1109/MSP.2011.2179329

In [ ]:
ssh_b = swotd.SWOTdenoise(ssh=ssh, lon=lon, lat=lat, x_ac=x_ac, time=time, method='bilateral_filter', param=(4,0.04) )
splot(ssh_b)